# Natural Language Processing

This is the filed of ML that focuses on creating models from a text data sources. That means all articles and availabe words!

This is extremaly large field of ML! We are just covering the small portion of it. 

More information can be found at Wikipedia Article on NLP. NLTK Book (Python library) that is not contained in PySpark. 
And finally the Foundations of Statistical Natural Language Processing (Manning). 

Examples of NLP: 

- Clustering News Articles
- Suggesting similair books
- Grouping Legal Docs
- Analyzing Consumer Feedback
- Spam Email Detection 

Basic process for NLP task concerns: 

- Compile all documents (Corpus)
- Featurize the words to numerics
- Compare features of documents

We can distingush more complex or complicated ML algorithms like Words to Vector or WordToVec that is analyzing every sentence separetely. 
We are focusing on classic NLP here.

The mechanism behind the NLP is based on TF-IDF method. 
That stands from Term Frequency - Inverse Document Frequency.

Basicly it is creating "Bag of Words". Meaning it creates a word count vector array from the number of words showing in the particular doc. Again a bag of counted words, the most simple :) 

We can improve on Bag of Words by adjusting word counts based on their frequency in corpus(the group of all thedocuments). 

We can however improve on the TF-IDF method a lot!

We end up doing is taking: 

- Term Frequency - Importance of the termin within the documnent. 
 - TF(x,y) = Number of occurences of term x in document y
 
- Inverse Document Frequency - Taking the length of doc into account.
    - n total is a total number of docs
    - the dfx stands for number of docs with the searched term. 
    - to achive the IDF(t) - we need to take the sum of n divided by dfx logarytmic value. 
    
To sum it up the w = tf with x and y. Times logarythmic value of N divided by df. So the TF-IDF. 

In simple words we are using some tool to caount the values in the document. What we are taking into account is: 
how often particular term is shows up in the document. And then using the inverse document frequency the actuall differences 
between document lenghts.



# Spam detection Model

In [1]:
import findspark
findspark.init('/home/ubuntu/Spark/spark-3.3.0-bin-hadoop3')

In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('NLP').getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/09/23 11:01:16 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
from pyspark.ml.feature import Tokenizer, RegexTokenizer

In [5]:
from pyspark.sql.functions import col, udf

In [7]:
from pyspark.sql.types import IntegerType

In [22]:
sen_df = spark.createDataFrame([
    (0, 'Hi I heard about Spark'),
    (1, 'I wish Java could use case classes'),
    (2, 'Logistic,Regression,models,are,neat'),
    
], ['id', 'sentence'])

In [23]:
sen_df.show()

+---+--------------------+
| id|            sentence|
+---+--------------------+
|  0|Hi I heard about ...|
|  1|I wish Java could...|
|  2|Logistic,Regressi...|
+---+--------------------+



In [24]:
tokenizer = Tokenizer(inputCol='sentence', outputCol='words')

In [25]:
regex_tokenzier = RegexTokenizer(inputCol='sentence', outputCol='words', pattern='\\W')

In [26]:
count_tokens = udf(lambda words:len(words), IntegerType())

In [27]:
tokenized = tokenizer.transform(sen_df)

In [28]:
tokenized.show()

+---+--------------------+--------------------+
| id|            sentence|               words|
+---+--------------------+--------------------+
|  0|Hi I heard about ...|[hi, i, heard, ab...|
|  1|I wish Java could...|[i, wish, java, c...|
|  2|Logistic,Regressi...|[logistic,regress...|
+---+--------------------+--------------------+



In [29]:
tokenized.withColumn('tokens', count_tokens(col("words"))).show()

+---+--------------------+--------------------+------+
| id|            sentence|               words|tokens|
+---+--------------------+--------------------+------+
|  0|Hi I heard about ...|[hi, i, heard, ab...|     5|
|  1|I wish Java could...|[i, wish, java, c...|     7|
|  2|Logistic,Regressi...|[logistic,regress...|     1|
+---+--------------------+--------------------+------+



In [30]:
rg_tokenzie = regex_tokenzier.transform(sen_df)

In [31]:
rg_tokenzie.withColumn('tokens', count_tokens(col("words"))).show()

+---+--------------------+--------------------+------+
| id|            sentence|               words|tokens|
+---+--------------------+--------------------+------+
|  0|Hi I heard about ...|[hi, i, heard, ab...|     5|
|  1|I wish Java could...|[i, wish, java, c...|     7|
|  2|Logistic,Regressi...|[logistic, regres...|     5|
+---+--------------------+--------------------+------+



In [32]:
from pyspark.ml.feature import StopWordsRemover

In [33]:
sentenceDataFrame = spark.createDataFrame([
    (0, ["I", "saw", "green", "horse"]),
    (1, ["Mary", "had", "a", "little", "lamb"])
    
], ['id', 'tokens'])

In [34]:
sentenceDataFrame.show()

+---+--------------------+
| id|              tokens|
+---+--------------------+
|  0|[I, saw, green, h...|
|  1|[Mary, had, a, li...|
+---+--------------------+



In [35]:
remover = StopWordsRemover(inputCol='tokens', outputCol='filtered')

In [36]:
remover.transform(sentenceDataFrame).show()

+---+--------------------+--------------------+
| id|              tokens|            filtered|
+---+--------------------+--------------------+
|  0|[I, saw, green, h...| [saw, green, horse]|
|  1|[Mary, had, a, li...|[Mary, little, lamb]|
+---+--------------------+--------------------+



In [37]:
# n-gram - sequence of tokens for a integer

In [39]:
from pyspark.ml.feature import NGram

In [52]:
wordDataFrame = spark.createDataFrame([
    (0, ['Hi','I','heard','about', 'Spark']),
    (1, ['I','wish','Java','could','use','case','classes']),
    (2, ['Logistic','Regression','models','are','neat']),
    
], ['id', 'words'])

In [53]:
ngram = NGram(n=2, inputCol='words', outputCol='grams')

In [54]:
ngram.transform(wordDataFrame).show()

+---+--------------------+--------------------+
| id|               words|               grams|
+---+--------------------+--------------------+
|  0|[Hi, I, heard, ab...|[Hi I, I heard, h...|
|  1|[I, wish, Java, c...|[I wish, wish Jav...|
|  2|[Logistic, Regres...|[Logistic Regress...|
+---+--------------------+--------------------+



In [56]:
ngram.transform(wordDataFrame).select('grams').show()

+--------------------+
|               grams|
+--------------------+
|[Hi I, I heard, h...|
|[I wish, wish Jav...|
|[Logistic Regress...|
+--------------------+



In [57]:
ngram.transform(wordDataFrame).select('grams').show(truncate=False)

+------------------------------------------------------------------+
|grams                                                             |
+------------------------------------------------------------------+
|[Hi I, I heard, heard about, about Spark]                         |
|[I wish, wish Java, Java could, could use, use case, case classes]|
|[Logistic Regression, Regression models, models are, are neat]    |
+------------------------------------------------------------------+

